# Detect entire series anomalies using Azure Cognitive Services (Anomaly Detector API)

In [1]:
# Import Libraries

import os
from azure.ai.anomalydetector import AnomalyDetectorClient
from azure.ai.anomalydetector.models import DetectRequest, TimeSeriesPoint, TimeGranularity, AnomalyDetectorError
from azure.core.credentials import AzureKeyCredential
import pandas as pd
import numpy as np
from msrest.authentication import CognitiveServicesCredentials

In [2]:
# Create variables for your key as an environment variable, 
# the path to a time series data file 
# and the Azure location of your subscription

SUBSCRIPTION_KEY = "f543655049434464bcddb3e63377e89c"
ANOMALY_DETECTOR_ENDPOINT = "https://anomaly-20-9.cognitiveservices.azure.com/"
#ANOMALY_DETECTOR_ENDPOINT = "https://anomaly-20-9.cognitiveservices.azure.com/anomalydetector/v1.0/timeseries/entire/detect"
TIME_SERIES_DATA_PATH = os.path.join("./", "timeSeries_split.csv")

In [3]:
# Create an Anomaly Detector client

client = AnomalyDetectorClient(AzureKeyCredential(SUBSCRIPTION_KEY), ANOMALY_DETECTOR_ENDPOINT)
#client = AnomalyDetectorClient(CognitiveServicesCredentials(SUBSCRIPTION_KEY), ANOMALY_DETECTOR_ENDPOINT)


In [4]:
# Load in the time series data file

series = []
data_file = pd.read_csv(TIME_SERIES_DATA_PATH, header=0, encoding='utf-8', date_parser=[0])


In [5]:

#data_file.columns = data_file.iloc[0]
#data_file = data_file[1:]

In [6]:
data_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Datetime  89 non-null     object
 1   Value     89 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [7]:
data_file

,Datetime,Value
0,2004-10-01T01:00:00Z,12379
1,2004-10-01T02:00:00Z,11935
2,2004-10-01T03:00:00Z,11692
3,2004-10-01T04:00:00Z,11597
4,2004-10-01T05:00:00Z,11681
...,...,...
84,2004-10-04T13:00:00Z,15662
85,2004-10-04T14:00:00Z,15803
86,2004-10-04T15:00:00Z,15747
87,2004-10-04T16:00:00Z,15689


In [8]:
for index, row in data_file.iterrows():
    series.append(TimeSeriesPoint(timestamp=row[0], value=row[1]))

In [9]:
# Create a DetectRequest object with your time series
# and the TimeGranularity (or periodicity) of its data points

request = DetectRequest(series=series, granularity=TimeGranularity.hourly)
#request = DetectRequest(series=np.take(series, [0, 8000], axis = 1)), granularity=TimeGranularity.daily)

In [10]:
len(series)

89

In [11]:
for i in range(0, len(series)):

     print(series[i])

{'additional_properties': {}, 'timestamp': '2004-10-01T01:00:00Z', 'value': 12379}
{'additional_properties': {}, 'timestamp': '2004-10-01T02:00:00Z', 'value': 11935}
{'additional_properties': {}, 'timestamp': '2004-10-01T03:00:00Z', 'value': 11692}
{'additional_properties': {}, 'timestamp': '2004-10-01T04:00:00Z', 'value': 11597}
{'additional_properties': {}, 'timestamp': '2004-10-01T05:00:00Z', 'value': 11681}
{'additional_properties': {}, 'timestamp': '2004-10-01T06:00:00Z', 'value': 12280}
{'additional_properties': {}, 'timestamp': '2004-10-01T07:00:00Z', 'value': 13692}
{'additional_properties': {}, 'timestamp': '2004-10-01T08:00:00Z', 'value': 14618}
{'additional_properties': {}, 'timestamp': '2004-10-01T09:00:00Z', 'value': 14903}
{'additional_properties': {}, 'timestamp': '2004-10-01T10:00:00Z', 'value': 15118}
{'additional_properties': {}, 'timestamp': '2004-10-01T11:00:00Z', 'value': 15242}
{'additional_properties': {}, 'timestamp': '2004-10-01T12:00:00Z', 'value': 15375}
{'ad

In [12]:
# Detect anomalies throughout the entire time series, as a batch

print('Detecting anomalies in the entire time series.')

try:
    response = client.detect_entire_series(request)
except AnomalyDetectorError as e:
    print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
except Exception as e:
    print(e)

if any(response.is_anomaly):
    print('An anomaly was detected at index:')
    for i, value in enumerate(response.is_anomaly):
        if value:
            print(i)
else:
    print('No anomalies were detected in the time series.')

Detecting anomalies in the entire time series.
An anomaly was detected at index:
12
13
14
15
16
17
18
30
31
32
33
54
55
56
57
58
59
60
61
62
63
64


In [13]:
# Detect the anomaly status of the latest data point

print('Detecting the anomaly status of the latest data point.')

try:
    response = client.detect_last_point(request)
except AnomalyDetectorError as e:
    print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
except Exception as e:
    print(e)

if response.is_anomaly:
    print('The latest point is detected as anomaly.')
else:
    print('The latest point is not detected as anomaly.')

Detecting the anomaly status of the latest data point.
The latest point is not detected as anomaly.


In [14]:
# Detect change points in the data set

print('Detecting change points in the entire time series.')

try:
    response = client.detect_change_point(request)
except AnomalyDetectorError as e:
    print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
except Exception as e:
    print(e)

if any(response.is_change_point):
    print('An change point was detected at index:')
    for i, value in enumerate(response.is_change_point):
        if value:
            print(i)
else:
    print('No change point were detected in the time series.')

Detecting change points in the entire time series.
No change point were detected in the time series.
